In [1]:
!pip install parallelPermutationTest
# parallelPermutationTest-1.0.3

import torch
import parallelPermutationTest as ppt
n_bins = 1000
import glob
import numpy as np
import pandas as pd
from google.colab import files
import random



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for parallelPermutationTest: filename=parallelPermutationTest-1.0.3-cp38-cp38-linux_x86_64.whl size=62569 sha256=c07fdf75b2ea33865926e02dcfacc8210f594d129444d4b9f7487cdd1e5dc8cc
  Stored in directory: /root/.cache/pip/wheels/0c/10/3d/d5202be9f597792669f4fb0da1ea9b80cebda7b7f33235faf2
Successfully built parallelPermutationTest


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Permutation Tests to calculate p-values

## Start with proliferation and differentiation

In [3]:
fdir = 'drive/MyDrive/colab_files/20221209_rerun_all/'
f_means = glob.glob(fdir+'*collate*data.csv')
print(f_means)
df = pd.read_csv(f_means[0])
df = df[df.exp.isin(['growth', 'differentiation'])]
df.exp.unique()

['drive/MyDrive/colab_files/20221209_rerun_all/collated_screen_data.csv']


array(['growth'], dtype=object)

In [4]:
for f, _df in df.groupby('exp'):
  df_genes_perm = pd.DataFrame()
  print(f)

  df_ctrl = _df[_df.gene.str.contains('CONTROL')]
  control = df_ctrl.log2fold_diff_mean.values 
  control = control[~np.isnan(control)]

  for gene, d in _df[~_df.gene.str.contains('CONTROL')].groupby('gene'):
    pvalue = []
    treatment = d.log2fold_diff_mean.values 

    if len(treatment) < 1:
      df_genes_perm = df_genes_perm.append({'gene' : gene,
                                      'log2fold_diff_mean' : d.log2fold_diff_mean.mean(),
                                      'exp' : f,
                                      'pvalue' : np.nan}, ignore_index=True)

    else:
      while True:
        try:
          pvalue = ppt.GreenFloatCuda(treatment, control, n_bins, batch_size=1)
          break
        except:
          break
      if len(pvalue) == 0:
        pvalue = [np.nan]
      df_genes_perm = df_genes_perm.append({'gene' : gene,
                                            'log2fold_diff_mean' : d.log2fold_diff_mean.mean(),
                                            'exp' : f,
                                            'pvalue' : pvalue[0]}, ignore_index=True)
      
  # Calculate control genes averages using 3 sgRNA per each control 'gene'
  d_ctrl = df_ctrl.copy().reset_index()
  for i in np.arange(2000):
    rndi = [random.randint(0,len(d_ctrl)) for k in np.arange(3)]
    _d = d_ctrl.loc[[random.randint(0,len(d_ctrl)-1) for k in np.arange(3)]] 
    pvalue = ppt.GreenFloatCuda(_d.log2fold_diff_mean.values, control, n_bins, batch_size=1)
    data_list = {'gene' : 'CONTROL_' + str(i),
            'log2fold_diff_mean' : _d.log2fold_diff_mean.mean(),
            'exp' : f,
            'pvalue' : pvalue[0]}
    df_genes_perm = df_genes_perm.append(data_list, ignore_index = True)

  df_genes_perm.to_csv('drive/MyDrive/colab_files/20221209_rerun_all/20220516_screen_log2fold_diffs_'+f+'_means_pvalue.csv')

growth


KeyboardInterrupt: ignored

## Now for cell migration experiments

In [18]:
fdir = 'drive/MyDrive/colab_files/20221209_rerun_all/'
f_means = glob.glob(fdir+'*collate*data.csv')
print(f_means)
df = pd.read_csv(f_means[0])
df = df[~df.exp.isin(['growth', 'differentiation'])]

['drive/MyDrive/colab_files/20221209_rerun_all/collated_screen_data.csv']


array(['tracketch_0_2hr_top', 'tracketch_0_2hr_bottom',
       'tracketch_0_6hr_top', 'tracketch_0_6hr_bottom',
       'tracketch_10_2hr_top', 'tracketch_10_2hr_bottom',
       'tracketch_10_6hr_top', 'tracketch_10_6hr_bottom', '3D_amoeboid'],
      dtype=object)

In [22]:
for f in ['tracketch_0_2hr', 'tracketch_0_6hr', 'tracketch_10_2hr', 'tracketch_10_6hr', '3D_amoeboid']:
  _df = df[df.exp.str.contains(f)]
  df_genes_perm = pd.DataFrame()

  print(f)

  df_ctrl = pd.DataFrame()
  control = []
  for sg, d in _df[_df.gene.str.contains('CONTROL')].groupby('gene'):
    control = np.append(control, d.log2fold_diff_mean.mean()) 
    datalist = {'gene' : sg,
                'log2fold_diff_mean' : d.log2fold_diff_mean.mean()}
    df_ctrl = df_ctrl.append(datalist, ignore_index = True)     
  control = control[~np.isnan(control)]


  for gene, d in _df[~_df.gene.str.contains('CONTROL')].groupby('gene'):
    pvalue = []

    treatment = d.groupby('sgRNA').log2fold_diff_mean.mean().values

    if len(treatment) < 1:
      df_genes_perm = df_genes_perm.append({'gene' : gene,
                                      'log2fold_diff_mean' : d.log2fold_diff_mean.mean(),
                                      'exp' : f,
                                      'pvalue' : np.nan}, ignore_index=True)

    else:
      while True:
        try:
          pvalue = ppt.GreenFloatCuda(treatment, control, n_bins, batch_size=1)
          break
        except:
          break
      if len(pvalue) == 0:
        pvalue = [np.nan]
      df_genes_perm = df_genes_perm.append({'gene' : gene,
                                            'log2fold_diff_mean' : d.log2fold_diff_mean.mean(),
                                            'exp' : f,
                                            'pvalue' : pvalue[0]}, ignore_index=True)
      
    
  # Calculate pseudogenes
  d_ctrl = df_ctrl.copy().reset_index()
  for i in np.arange(2000):
    rndi = [random.randint(0,len(d_ctrl)) for k in np.arange(3)]
    _d = d_ctrl.loc[[random.randint(0,len(d_ctrl)-1) for k in np.arange(3)]] 
    pvalue = ppt.GreenFloatCuda(_d.log2fold_diff_mean.values, control, n_bins, batch_size=1)
    data_list = {'gene' : 'CONTROL_' + str(i),
            'log2fold_diff_mean' : _d.log2fold_diff_mean.mean(),
            'exp' : f,
            'pvalue' : pvalue[0]}
    df_genes_perm = df_genes_perm.append(data_list, ignore_index = True)

  df_genes_perm.to_csv('drive/MyDrive/colab_files/20221209_rerun_all/20220516_screen_log2fold_diffs_'+f+'_means_pvalue_2.csv')

tracketch_0_2hr
tracketch_0_6hr
tracketch_10_2hr
tracketch_10_6hr
3D_amoeboid


## Let's also look at all migration data

In [29]:
fdir = 'drive/MyDrive/colab_files/20221209_rerun_all/'
f_means = glob.glob(fdir+'*collate*data.csv')
print(f_means)
df = pd.read_csv(f_means[0])
_df = df[~df.exp.isin(['growth', 'differentiation'])]
_df.exp.unique()

['drive/MyDrive/colab_files/20221209_rerun_all/collated_screen_data.csv']


array(['tracketch_0_2hr_top', 'tracketch_0_2hr_bottom',
       'tracketch_0_6hr_top', 'tracketch_0_6hr_bottom',
       'tracketch_10_2hr_top', 'tracketch_10_2hr_bottom',
       'tracketch_10_6hr_top', 'tracketch_10_6hr_bottom', '3D_amoeboid'],
      dtype=object)

In [30]:
df_genes_perm = pd.DataFrame()
f = 'migration_all'
print(f)

df_ctrl = pd.DataFrame()
control = []
for sg, d in _df[_df.gene.str.contains('CONTROL')].groupby('gene'):
  control = np.append(control, d.log2fold_diff_mean.mean()) 
  datalist = {'gene' : sg,
              'log2fold_diff_mean' : d.log2fold_diff_mean.mean()}
  df_ctrl = df_ctrl.append(datalist, ignore_index = True)     
control = control[~np.isnan(control)]


for gene, d in _df[~_df.gene.str.contains('CONTROL')].groupby('gene'):
  pvalue = []

  treatment = d.groupby('sgRNA').log2fold_diff_mean.mean().values

  if len(treatment) < 1:
    df_genes_perm = df_genes_perm.append({'gene' : gene,
                                    'log2fold_diff_mean' : d.log2fold_diff_mean.mean(),
                                    'exp' : f,
                                    'pvalue' : np.nan}, ignore_index=True)

  else:
    while True:
      try:
        pvalue = ppt.GreenFloatCuda(treatment, control, n_bins, batch_size=1)
        break
      except:
        break
    if len(pvalue) == 0:
      pvalue = [np.nan]
    df_genes_perm = df_genes_perm.append({'gene' : gene,
                                          'log2fold_diff_mean' : d.log2fold_diff_mean.mean(),
                                          'exp' : f,
                                          'pvalue' : pvalue[0]}, ignore_index=True)
    
  
# Calculate pseudogenes
d_ctrl = df_ctrl.copy().reset_index()
for i in np.arange(2000):
  rndi = [random.randint(0,len(d_ctrl)) for k in np.arange(3)]
  _d = d_ctrl.loc[[random.randint(0,len(d_ctrl)-1) for k in np.arange(3)]] 
  pvalue = ppt.GreenFloatCuda(_d.log2fold_diff_mean.values, control, n_bins, batch_size=1)
  data_list = {'gene' : 'CONTROL_' + str(i),
          'log2fold_diff_mean' : _d.log2fold_diff_mean.mean(),
          'exp' : f,
          'pvalue' : pvalue[0]}
  df_genes_perm = df_genes_perm.append(data_list, ignore_index = True)

df_genes_perm.to_csv('drive/MyDrive/colab_files/20221209_rerun_all/20220516_screen_log2fold_diffs_'+f+'_means_pvalue_2.csv')

migration_all


### Now, let's also repeat for a couple additional files: Combined chemokinesis (2 hr and 6 hr), combined chemotaxis (2 hr and 6 hr), and combined chemokinesis + chemotaxis. Here we will treat all exps as individual replicates.

In [ ]:
fdir = 'drive/MyDrive/colab_files/20221209_rerun_all/'
f_means = glob.glob(fdir+'*collate*misc.csv')
df = pd.read_csv(f_means[0])
df.exp.unique()

In [ ]:
for f in ['tracketch_0', 'tracketch_10']:
  _df = df[df.exp.str.contains(f)]
  df_genes_perm = pd.DataFrame()

  print(f)

  df_ctrl = pd.DataFrame()
  control = []
  for sg, d in _df[_df.gene.str.contains('CONTROL')].groupby('gene'):
    control = np.append(control, d.log2fold_diff_mean.mean()) 
    datalist = {'gene' : sg,
                'log2fold_diff' : d.log2fold_diff_mean.mean()}
    df_ctrl = df_ctrl.append(datalist, ignore_index = True)     
  control = control[~np.isnan(control)]


  for gene, d in _df[~_df.gene.str.contains('CONTROL')].groupby('gene'):
    pvalue = []

    treatment = d.groupby('sgRNA').log2fold_diff_mean.mean().values

    if len(treatment) < 1:
      df_genes_perm = df_genes_perm.append({'gene' : gene,
                                      'log2fold_diff_mean' : d.log2fold_diff_mean.mean(),
                                      'exp' : f,
                                      'pvalue' : np.nan}, ignore_index=True)

    else:
      while True:
        try:
          pvalue = ppt.GreenFloatCuda(treatment, control, n_bins, batch_size=1)
          break
        except:
          break
      if len(pvalue) == 0:
        pvalue = [np.nan]
      df_genes_perm = df_genes_perm.append({'gene' : gene,
                                            'log2fold_diff_mean' : d.log2fold_diff_mean.mean(),
                                            'exp' : f,
                                            'pvalue' : pvalue[0]}, ignore_index=True)
      
    
  # Calculate pseudogenes
  d_ctrl = df_ctrl.copy().reset_index()
  for i in np.arange(2000):
    rndi = [random.randint(0,len(d_ctrl)) for k in np.arange(3)]
    _d = d_ctrl.loc[[random.randint(0,len(d_ctrl)-1) for k in np.arange(3)]] 
    pvalue = ppt.GreenFloatCuda(_d.log2fold_diff_mean.values, control, n_bins, batch_size=1)
    data_list = {'gene' : 'CONTROL_' + str(i),
            'log2fold_diff_mean' : _d.log2fold_diff_mean.mean(),
            'exp' : f,
            'pvalue' : pvalue[0]}
    df_genes_perm = df_genes_perm.append(data_list, ignore_index = True)

  df_genes_perm.to_csv('drive/MyDrive/colab_files/20221209_rerun_all/20220516_screen_log2fold_diffs_'+f+'_means_pvalue.csv')

In [ ]:
# fdir = 'drive/MyDrive/colab_files/20221209_rerun_all/'
# f_means = glob.glob(fdir+'*collate*misc.csv')
# df = pd.read_csv(f_means[0])
# df.exp.unique()

In [ ]:
# for f, _df in df.groupby('exp'):
#   df_genes_perm = pd.DataFrame()
#   print(f)

#   df_ctrl = _df[_df.gene.str.contains('CONTROL')]
#   control = df_ctrl.log2fold_diff_mean.values 
#   control = control[~np.isnan(control)]

#   for gene, d in _df[~_df.gene.str.contains('CONTROL')].groupby('gene'):
#     pvalue = []
#     treatment = d.log2fold_diff_mean.values 

#     if len(treatment) < 1:
#       df_genes_perm = df_genes_perm.append({'gene' : gene,
#                                       'log2fold_diff_mean' : d.log2fold_diff_mean.mean(),
#                                       'exp' : f,
#                                       'pvalue' : np.nan}, ignore_index=True)

#     else:
#       while True:
#         try:
#           pvalue = ppt.GreenFloatCuda(treatment, control, n_bins, batch_size=1)
#           break
#         except:
#           break
#       if len(pvalue) == 0:
#         pvalue = [np.nan]
#       df_genes_perm = df_genes_perm.append({'gene' : gene,
#                                             'log2fold_diff_mean' : d.log2fold_diff_mean.mean(),
#                                             'exp' : f,
#                                             'pvalue' : pvalue[0]}, ignore_index=True)
      
#   # Calculate control genes averages using 3 sgRNA per each control 'gene'
#   d_ctrl = df_ctrl.copy().reset_index()
#   for i in np.arange(2000):
#     rndi = [random.randint(0,len(d_ctrl)) for k in np.arange(3)]
#     _d = d_ctrl.loc[[random.randint(0,len(d_ctrl)-1) for k in np.arange(3)]] 
#     pvalue = ppt.GreenFloatCuda(_d.log2fold_diff_mean.values, control, n_bins, batch_size=1)
#     data_list = {'gene' : 'CONTROL_' + str(i),
#             'log2fold_diff_mean' : _d.log2fold_diff_mean.mean(),
#             'exp' : f,
#             'pvalue' : pvalue[0]}
#     df_genes_perm = df_genes_perm.append(data_list, ignore_index = True)

#   df_genes_perm.to_csv('drive/MyDrive/colab_files/20221209_rerun_all/20220516_screen_log2fold_diffs_'+f+'_means_pvalue.csv')

## Correct some erroneous permutation tests. 

Some of the comparisons where the effect size is quite large result in a 0 or non-computed p-value. Here I'm going to give them a value of the minimum p-value that could be calculated across the other permutation tests.

In [31]:
# For gene averaged values
files = glob.glob('drive/MyDrive/colab_files/20221209_rerun_all/20220516_screen_log2fold_diffs_*_means_pvalue_2.csv')
files

['drive/MyDrive/colab_files/20221209_rerun_all/20220516_screen_log2fold_diffs_tracketch_0_2hr_means_pvalue_2.csv',
 'drive/MyDrive/colab_files/20221209_rerun_all/20220516_screen_log2fold_diffs_tracketch_0_6hr_means_pvalue_2.csv',
 'drive/MyDrive/colab_files/20221209_rerun_all/20220516_screen_log2fold_diffs_tracketch_10_2hr_means_pvalue_2.csv',
 'drive/MyDrive/colab_files/20221209_rerun_all/20220516_screen_log2fold_diffs_tracketch_10_6hr_means_pvalue_2.csv',
 'drive/MyDrive/colab_files/20221209_rerun_all/20220516_screen_log2fold_diffs_3D_amoeboid_means_pvalue_2.csv',
 'drive/MyDrive/colab_files/20221209_rerun_all/20220516_screen_log2fold_diffs_migration_all_means_pvalue_2.csv']

In [32]:
for f in files:
  print(f)
  df = pd.read_csv(f)
  pvalue_min = df.pvalue.min()

  df_nan = df[df.pvalue.isnull()]
  df = df[~df.pvalue.isnull()]
  if ~df_nan.empty:
    df_nan['pvalue'] = pvalue_min

  df = df.append(df_nan, ignore_index = True)
  
  df.to_csv(f.split('.csv')[0]+'_.csv')

drive/MyDrive/colab_files/20221209_rerun_all/20220516_screen_log2fold_diffs_tracketch_0_2hr_means_pvalue_2.csv
drive/MyDrive/colab_files/20221209_rerun_all/20220516_screen_log2fold_diffs_tracketch_0_6hr_means_pvalue_2.csv
drive/MyDrive/colab_files/20221209_rerun_all/20220516_screen_log2fold_diffs_tracketch_10_2hr_means_pvalue_2.csv
drive/MyDrive/colab_files/20221209_rerun_all/20220516_screen_log2fold_diffs_tracketch_10_6hr_means_pvalue_2.csv
drive/MyDrive/colab_files/20221209_rerun_all/20220516_screen_log2fold_diffs_3D_amoeboid_means_pvalue_2.csv
drive/MyDrive/colab_files/20221209_rerun_all/20220516_screen_log2fold_diffs_migration_all_means_pvalue_2.csv


In [11]:
# For sgRNA averaged values
files = glob.glob('drive/MyDrive/colab_files/20221209_rerun_all/20220516_screen_log2fold_diffs_*_sgRNA_pvalue.csv')
files

[]

In [12]:
for f in files:
  print(f)
  df = pd.read_csv(f)
  pvalue_min = df.pvalue.min()

  df_nan = df[df.pvalue.isnull()]
  df = df[~df.pvalue.isnull()]
  if ~df_nan.empty:
    df_nan['pvalue'] = pvalue_min

  df = df.append(df_nan, ignore_index = True)
  
  df.to_csv(f.split('.csv')[0]+'_.csv')

## Lets assess sgRNAs across all the experiments.

I'm aware of a number of genes where  there only appear to be 1/3 of the sgRNA that is effective. However it's a bit unclear how to assess and distinguish those from noise. In any case, I want to also quantify the individual sgRNA, particularly for plots and to help identify the strongest sgRNA perturbations. Lets repeat above, but with individual sgRNAs.

In [ ]:
# files containing log2foldchange data
fdir = 'drive/MyDrive/colab_files/20221209_rerun_all/'
f_means = glob.glob(fdir+'*collate*misc.csv')

df = pd.read_csv(f_means[0])
df.exp.unique()

array(['tracketch_0_all', 'tracketch_10_all', 'tracketch_all',
       'migration_all'], dtype=object)

In [ ]:
for f, _df in df.groupby('exp'):
  df_genes_perm = pd.DataFrame()
  print(f)

  df_ctrl = _df[_df.gene.str.contains('CONTROL')]
  control = df_ctrl.log2fold_diff_mean.values 
  control = control[~np.isnan(control)]

  for sg, d in _df[~_df.gene.str.contains('CONTROL')].groupby('sgRNA'):
    pvalue = []
    treatment = d.log2fold_diff_mean.values 

    if len(treatment) < 1:
      df_genes_perm = df_genes_perm.append({'gene' : d.gene.values[0],
                                            'sgRNA' : sg,
                                      'log2fold_diff_mean' : d.log2fold_diff_mean.mean(),
                                      'exp' : f,
                                      'pvalue' : np.nan}, ignore_index=True)

    else:
      while True:
        try:
          pvalue = ppt.GreenFloatCuda(treatment, control, n_bins, batch_size=1)
          break
        except:
          break
      if len(pvalue) == 0:
        pvalue = [np.nan]
      df_genes_perm = df_genes_perm.append({'gene' : d.gene.values[0],
                                            'sgRNA' : sg,
                                            'log2fold_diff_mean' : d.log2fold_diff_mean.mean(),
                                            'exp' : f,
                                            'pvalue' : pvalue[0]}, ignore_index=True)
      


  df_genes_perm.to_csv('drive/MyDrive/colab_files/20221209_rerun_all/20220516_screen_log2fold_diffs_'+f+'_sgRNA_pvalue.csv')

migration_all
tracketch_0_all
tracketch_10_all
tracketch_all


In [6]:
# files containing log2foldchange data
fdir = 'drive/MyDrive/colab_files/20221209_rerun_all/'
f_means = glob.glob(fdir+'*collate*data.csv')

df = pd.read_csv(f_means[0])
df = df[df.exp == '3D_amoeboid']

In [7]:
for f, _df in df.groupby('exp'):
  df_genes_perm = pd.DataFrame()
  print(f)

  df_ctrl = _df[_df.gene.str.contains('CONTROL')]
  control = df_ctrl.log2fold_diff_mean.values 
  control = control[~np.isnan(control)]

  for sg, d in _df[~_df.gene.str.contains('CONTROL')].groupby('sgRNA'):
    pvalue = []
    treatment = d.log2fold_diff_mean.values 

    if len(treatment) < 1:
      df_genes_perm = df_genes_perm.append({'gene' : d.gene.values[0],
                                            'sgRNA' : sg,
                                      'log2fold_diff_mean' : d.log2fold_diff_mean.mean(),
                                      'exp' : f,
                                      'pvalue' : np.nan}, ignore_index=True)

    else:
      while True:
        try:
          pvalue = ppt.GreenFloatCuda(treatment, control, n_bins, batch_size=1)
          break
        except:
          break
      if len(pvalue) == 0:
        pvalue = [np.nan]
      df_genes_perm = df_genes_perm.append({'gene' : d.gene.values[0],
                                            'sgRNA' : sg,
                                            'log2fold_diff_mean' : d.log2fold_diff_mean.mean(),
                                            'exp' : f,
                                            'pvalue' : pvalue[0]}, ignore_index=True)
      


  df_genes_perm.to_csv('drive/MyDrive/colab_files/20221209_rerun_all/20220516_screen_log2fold_diffs_'+f+'_sgRNA_pvalue.csv')

3D_amoeboid
